### Module 16: Lab: Even more PCA

You try it with an even wider dataset.

In [ ]:
# Load libraries
from sklearn import datasets, linear_model
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA
import boto3
import pickle 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Load the gene expression, very wide dataset
sess = boto3.session.Session()
s3 = sess.client('s3') 
source_bucket = 'machinelearning-read-only' 
source_key = 'data/gene_expression_wide.pkl'
#
# Get the file from S3 
response = s3.get_object(Bucket = source_bucket, Key = source_key)
#
# Read the 'Body' part of the response into a variable. This is where the DataFrame data exists in the response.
body = response['Body'].read()
#
# Create a new pandas DataFrame using the pickle.loads() function
df = pickle.loads(body)
print('Dataset size:', df.shape)
df.head()

### PCA

In [ ]:
# We are now only focused on the feature matrix:
X = df.drop(['target'], axis = 1)
print('Dataset size:', X.shape)
X.head(2)

### See how much variance is captured in each PC

In [2]:
# You decide how many features you can plot in the graph below. 
# There is a maximum that is much lower than 20,532 columns
max_features = 'your integer value here'

In [ ]:
# Setup a scaler-PCA pipeline
#
scaler = StandardScaler() # Standardize the data
# Create a PCA model
pca = PCA(n_components = max_features,random_state = 11) 
#
steps = [('Scaler', scaler), ('PCA', pca)]
# Create a pipeline
pipe = Pipeline(steps)
# Fit
pipe.fit(X)
# Transform
principle_array = pipe.transform(X)
# Build a df
principal_df = pd.DataFrame(data = principle_array)
#
# Show the captured variance curve
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Explained Variance')
plt.show()

### How many Principal Components will you include?
- Set the n_components to the minimum percent of variance you want to capture

In [ ]:
# You decide how much variance to capture:
min_var = 'Your decimal value here'

In [ ]:
# Setup a scaler-PCA pipeline
#
scaler = StandardScaler() # Standardize the data
# Create a PCA model
# Specify the percent of variance you want to capture
pca = PCA(n_components = min_var, random_state = 11) # make sure we discuss n_componets = .9
#
steps = [('Scaler', scaler), ('PCA', pca)]
# Create a pipeline
pipe = Pipeline(steps)
# Fit
pipe.fit(X)
# Transform
principle_array = pipe.transform(X)
# Build a df
pca_df = pd.DataFrame(data = principle_array)
# Reduced
print('Shape of new feature matrix:', pca_df.shape)
print('We captured this percent of variance:', round(sum(pca.explained_variance_ratio_),3))

### Perform machine learning process on reduced dataset


In [ ]:
# Features
X = pca_df # The reduced dataset
# Target: Did the patient have cancer?
y = df['target'] # Create a series
# Split into train/test
# Reserve 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20,random_state = 42)
# Verify the sizes of the split datasets
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('X_test:', X_test.shape)
print('y_test:', y_test.shape)

### Use a pipeline perform modeling

In [ ]:
# Trying to predict which type of cancer was detected
y_test.value_counts()

In [ ]:
# Use SVM Muticlass Classification
scaler = StandardScaler()
svm_mc = svm.SVC()
steps = [('Scaler', scaler), ('SVM', svm_mc)]
pipe = Pipeline(steps)
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
acc = svm_mc.score(X_test, y_test) # Number predicted correctly divided by total in data set
print('Our model accuracy:', round(acc,4))
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm,
                     index = ['PRAD', 'LUAD', 'BRCA', 'KIRC', 'COAD'], 
                     columns = ['PRAD', 'LUAD', 'BRCA', 'KIRC', 'COAD'])
cm_df

In [ ]:
# Let's discuss this output
# We can refer back to Notebook 14-1-SVM for comparison